# 5TF078 Deep Learning Course
## Excercise 4 NLP - Word2Vec
Created by Tomas Nordström, Umeå University

Revisions:
* 2023-12-03 Initial version with three different word2vec models (Builtin, Glove, GoogleNews) to be used from within GenSim /ToNo
* 2023-12-06 Fix for gensim.downloader that now seems missing /Tomas
* 2024-03-24 Updated tests for Kaggle. /Tomas
* 2024-04-23 Included student calculation of similarity /Tomas
* 2024-04-30 Fixed depreciated gensim glove2word2vec /Tomas

# Initialization

In [ ]:
import sys
import os

### Is this notebook running on Colab?
IS_COLAB = "google.colab" in sys.modules

### Is this notebook running on Kaggle?
# Fool Kaggle into making kaggle_secrets avaiable
try:
    import kaggle_secrets
except ImportError as e:
    pass
# Now we can test for Kaggle
IS_KAGGLE = "kaggle_secrets" in sys.modules

In [ ]:
os.environ["KERAS_BACKEND"] = "jax" # Also jax,pytorch for Keras 3.0

# Import Keras/TF libraries

import keras
print('Keras version:', keras.__version__)

import tensorflow as tf
print('TensorFlow version:', tf.__version__)

Keras version: 3.5.0
TensorFlow version: 2.17.1


In [ ]:
# Helper libraries
import time
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split

import urllib
from zipfile import ZipFile

import gensim
import gensim.downloader as gsapi

# Matlab plotting
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# https://stackoverflow.com/questions/37748105/how-to-use-progressbar-module-with-urlretrieve#53643011
import progressbar
class MyProgressBar():
    def __init__(self):
        self.pbar = None

    def __call__(self, block_num, block_size, total_size):
        if not self.pbar:
            self.pbar=progressbar.ProgressBar(maxval=total_size)
            self.pbar.start()

        downloaded = block_num * block_size
        if downloaded < total_size:
            self.pbar.update(downloaded)
        else:
            self.pbar.finish()

# Based on the Gensim framework

Gensim Docs: https://radimrehurek.com/gensim/

There are many ways to download models or data for word2vec models:
1. The Gensim Builtin models (where 'glove-twitter-100' downloads 387 MB file)
2. One of the Glove models (downloads a 822 MB zip file).
3. GoogleNews based model

In [ ]:
# Select one of Builtin, Glove, GoogleNews models to use
model_to_use            = 'GoogleNews'
model_to_use_if_builtin = 'glove-twitter-100'

# Set up a word2vec model

## Look at what builtin Gensim models we can use

Note that you need to select one out of the possible builtin models!

In [ ]:
# https://radimrehurek.com/gensim/models/word2vec.html

# Show all available models in gensim-data
print(list(gsapi.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
if model_to_use == 'Builtin':
  w2v = gsapi.load(model_to_use_if_builtin)

## Using GloVe

Let's download pre-trained GloVe embeddings (a 822 MB zip file).

The archive contains text-encoded vectors of various sizes: 50-dimensional, 100-dimensional, 200-dimensional, 300-dimensional. We'll use the 100-D ones.



In [ ]:
# Download a file to kerasdata, but first check if it already exist, now with progress bar
DOWNLOADS_DIR = './kerasdata'
os.makedirs(DOWNLOADS_DIR, exist_ok=True) # create dir if not exist

if model_to_use == 'Glove':
  url= 'https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip'
  # Split on the rightmost / and take everything on the right side of that; up until '?'
  name = url.rsplit('/', 1)[-1].split('?', 1)[0]
  filename = os.path.join(DOWNLOADS_DIR, name)

  # Download the file if it does not exist
  if not os.path.isfile(filename):
    print(f'Retrieving url: {url}', flush=True)
    urllib.request.urlretrieve(url, filename, reporthook=MyProgressBar())
  else:
    print(f'Using local zip file: {filename}')

  GLOVEDIR = os.path.join(DOWNLOADS_DIR, 'Glove')
  path_to_glove_file =  os.path.join(GLOVEDIR, 'glove.6B.100d.txt')
  if not os.path.isfile(path_to_glove_file):
    ZipFile(filename).extractall(GLOVEDIR)

  print(f'Using local glove file: {path_to_glove_file}')

  # Now convert the Glove file to a gensim word2vec
  # https://stackoverflow.com/questions/48743053/how-to-save-and-load-glove-models#51319383
  GENSIMGLOVEFILE = os.path.join(GLOVEDIR,"gensim_glove_vectors.txt")

  if not os.path.isfile(GENSIMGLOVEFILE):
    # glove2word2vec(glove_input_file=path_to_glove_file, word2vec_output_file=GENSIMGLOVEFILE)
    w2v = gensim.models.KeyedVectors.load_word2vec_format(path_to_glove_file, binary=False, no_header=True)
  else:
    w2v = gensim.models.KeyedVectors.load_word2vec_format(GENSIMGLOVEFILE, binary=False)


## Using GoogleNews file

In [ ]:
if model_to_use == 'GoogleNews':
  # Download a file to kerasdata, but first check if it already exist, now with progress bar
  DOWNLOADS_DIR = './kerasdata'
  os.makedirs(DOWNLOADS_DIR, exist_ok=True) # create dir if not exist

  url= 'https://git.ri.se/tomas.nordstrom/mldata/-/raw/main/GoogleNews-vectors-negative300.bin.gz?inline=false'
  # Split on the rightmost / and take everything on the right side of that; up until '?'
  name = url.rsplit('/', 1)[-1].split('?', 1)[0]
  filename = os.path.join(DOWNLOADS_DIR, name)

  # Download the file if it does not exist
  if not os.path.isfile(filename):
    print(f'Retrieving url: {url}', flush=True)
    urllib.request.urlretrieve(url, filename, reporthook=MyProgressBar())
  else:
    print(f'Using local file: {filename}')

  # Create the word2vec model
  w2v = gensim.models.KeyedVectors.load_word2vec_format(filename, binary=True)

Using local file: ./kerasdata/GoogleNews-vectors-negative300.bin.gz


# Experiments

Check out https://radimrehurek.com/gensim/models/keyedvectors.html#what-can-i-do-with-word-vectors for examples what you can do with these vectors.

In [ ]:
# First check out the word2vec model
print(f'Model name {model_to_use}')
print(f'Total number of words: {len(w2v.key_to_index)}') # Totalt antal ord
print(f'Some example words: {list(w2v.key_to_index)[:20]}')
embedding_len = len(w2v[0])
print(f'Embedding vector length: {embedding_len}')

# The check out a vector
word1 = "cat"
print(f'\nLooking at "{word1}"')
cat_vector = w2v[word1]
print(f'Vector: {cat_vector[:10]}...')

Model name GoogleNews
Total number of words: 3000000
Some example words: ['</s>', 'in', 'for', 'that', 'is', 'on', '##', 'The', 'with', 'said', 'was', 'the', 'at', 'not', 'as', 'it', 'be', 'from', 'by', 'are']
Embedding vector length: 300

Looking at "cat"
Vector: [ 0.0123291   0.20410156 -0.28515625  0.21679688  0.11816406  0.08300781
  0.04980469 -0.00952148  0.22070312 -0.12597656]...


## Similarity

Now we want to find similarity between word vectors and need to define a [similarity measure](https://en.wikipedia.org/wiki/Similarity_measure). In this exercise we will use [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity) as a similarity measure.

Your task is now to define a similarity function, and then generate three word vectors for “cat”, “cut”, and “dog” and compare the similarity between them using your similarity function.


In [ ]:
def similarity(vec1,vec2):
##################### TODO: YOUR CODE STARTS HERE #####################
    dot_product = np.dot(vec1, vec2)
    magnitude_vec1 = np.linalg.norm(vec1)
    magnitude_vec2 = np.linalg.norm(vec2)
    # Compute cosine similarity
    cosine_similarity = dot_product / (magnitude_vec1 * magnitude_vec2)
    return cosine_similarity
##################### TODO: YOUR CODE ENDS HERE #######################

In [ ]:
cat_vector = w2v["cat"]
cut_vector = w2v["cut"]
dog_vector = w2v["dog"]

##################### TODO: YOUR CODE STARTS HERE #####################
# Compare the similarity between "cat”, “cut”, and “dog”
# Compare similarities
similarity_cat_cut = similarity(cat_vector, cut_vector)
similarity_cat_dog = similarity(cat_vector, dog_vector)
similarity_cut_dog = similarity(cut_vector, dog_vector)

# Display the similarities
print(f'Similarity between "{"cat"}" and "{"cut"}": {similarity_cat_cut:.4f}')
print(f'Similarity between "{"cat"}" and "{"dog"}": {similarity_cat_dog:.4f}')
print(f'Similarity between "{"cut"}" and "{"dog"}": {similarity_cut_dog:.4f}')

##################### TODO: YOUR CODE ENDS HERE #######################

Similarity between "cat" and "cut": 0.0926
Similarity between "cat" and "dog": 0.7609
Similarity between "cut" and "dog": 0.0555


## Uppgift
Skapa fler exempel och analysera dina resultat

In [ ]:
##################### TODO: YOUR CODE STARTS HERE #####################
car_vector = w2v["car"]
tree_vector = w2v["tree"]
animal_vector = w2v["animal"]

##################### TODO: YOUR CODE STARTS HERE #####################
# Compare the similarity between "cat”, “cut”, and “dog”
# Compare similarities
similarity_car_tree = similarity(car_vector, tree_vector)
similarity_car_animal = similarity(car_vector, animal_vector)
similarity_tree_animal = similarity(tree_vector, animal_vector)

# Display the similarities
print(f'Similarity between "{"car"}" and "{"tree"}": {similarity_car_tree:.4f}')
print(f'Similarity between "{"car"}" and "{"animal"}": {similarity_car_animal:.4f}')
print(f'Similarity between "{"tree"}" and "{"animal"}": {similarity_tree_animal:.4f}')


##################### TODO: YOUR CODE ENDS HERE #######################

Similarity between "car" and "tree": 0.2711
Similarity between "car" and "animal": 0.1680
Similarity between "tree" and "animal": 0.2878


In the example, three new words are added: "car", "tree", and "animal", and their vector representations are calculated.

Semantically similar words (e.g., "dog" and "cat").
Semantically unrelated words (e.g., "car" and "tree").
Domain-specific words (e.g., words related to technology, animals, or nature).

Similarity Calculation:
The similarity function implemented earlier is used to compute the cosine similarity between these words.

The similarity values between each pair of words are displayed to help understand how the model captures relationships between the words.

## Finding most similar
We are also interested in finding the n closest words to a certain vector.

We could do this by calculating a similarity to all vectors in the word2vec dictionary/matrix and then sort according to similarity score and take the top n-values.

This is clearly doable, but for this course round we will use the most_similar methods on our w2v object.
To get the ten most similar words to computer we do:
`sims = w2v.most_similar('computer', topn=10)`


We sometimes want to do operations with the vectors to do “king”-”man”+”woman”, and to support that we can use the two parameters: positive and negative:
`w2v.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)`


In [ ]:
# Classic example
result = w2v.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

In [ ]:
##################### TODO: YOUR CODE STARTS HERE #####################
# Create more examples
example1 = w2v.most_similar('science', topn = 1)
print(example1)

example2 = w2v.most_similar(positive=['Paris', 'Italy'], negative=['France'], topn=1)
print(example2)

example3 = w2v.most_similar('computer', topn=3)
print(example3)

example4 = w2v.most_similar(positive=['man', 'queen'], negative=['woman'], topn=1)
print(example4)

example5 = w2v.most_similar(positive=['dog', 'cat'], negative=['animal'], topn=1)
print(example5)
##################### TODO: YOUR CODE ENDS HERE #######################

## Uppgift
Skapa fler "räkneexempel" och analysera dina resultat

## There is actually a builtin similarity method

In [ ]:
# There is a built in similarity methods we can use instead of our own function
word1 = 'cat'
word2 = 'dog'
print(f"Similarity between {word1} and {word2} is {w2v.similarity(word1, word2)}")

word2 = 'cut'
print(f"Similarity between {word1} and {word2} is {w2v.similarity(word1, word2)}")

## Uppgift
Får du samma resultat med din likhetsfunktion som den i gensim inbyggda funktionen?

1. This confirms that our implementation is mathematically correct from wikipedia.

2. Both methods calculate cosine similarity, which measures the angle between two vectors in a high-dimensional space. Since the formula is standard and deterministic, the results are the same.

3. While the custom function demonstrates your understanding of cosine similarity, the built-in method is more convenient and optimized for production use, as it avoids redundant calculations and directly interfaces with the word embeddings.